# GraphQL Ask Question Resolver - Interactive Tutorial
# ===================================================

In this notebook, you'll learn how to implement GraphQL mutations for asking questions in the RAG Engine.

## 📚 Learning Objectives

By the end of this notebook, you will:
- Understand GraphQL mutations
- Implement ask_question mutation
- Integrate with RAG use case
- Handle errors and validation
- Learn performance optimization techniques

## 🔧 Prerequisites

Ensure you have the following installed:
- Python 3.11+
- Strawberry GraphQL
- FastAPI
- RAG Engine dependencies

## 📦 Setup

Let's start by importing necessary libraries.

In [ ]:
# Import required libraries
import asyncio
from typing import List, Optional
from datetime import datetime
from dataclasses import dataclass
from unittest.mock import Mock

# GraphQL library
import strawberry

# Print setup confirmation
print("✅ Libraries imported successfully!")
print(f"   - Strawberry version: {strawberry.__version__}")

## 1. GraphQL Mutations

Mutations are GraphQL operations that modify data on the server. While queries fetch data (read-only), mutations perform write operations.

### 1.1 Mutation Definition

Let's define the Answer type and Mutation type.

In [ ]:
# Define GraphQL types for ask question

@strawberry.type
class AnswerType:
    """Answer GraphQL type with sources."""
    text: str
    sources: List[str]
    retrieval_k: int
    embed_ms: Optional[int]
    search_ms: Optional[int]
    llm_ms: Optional[int]

@strawberry.enum
class DocumentStatus(strawberry.Enum):
    CREATED = "created"
    INDEXED = "indexed"
    FAILED = "failed"

@strawberry.type
class DocumentType:
    id: strawberry.ID
    filename: str
    content_type: str
    size_bytes: int
    status: DocumentStatus
    created_at: datetime
    updated_at: Optional[datetime]

print("✅ GraphQL types defined successfully!")

### 1.2 Mock Use Case

Let's create a mock use case for the ask question operation.

In [ ]:
# Mock Answer dataclass
@dataclass
class MockAnswer:
    """Mock answer from use case."""
    text: str
    sources: List[str]
    retrieval_k: int
    embed_ms: Optional[int]
    search_ms: Optional[int]
    llm_ms: Optional[int]

# Mock AskQuestionHybridUseCase
class MockAskUseCase:
    """Mock use case for asking questions."""
    
    def execute(self, request):
        """Execute mock ask question."""
        # Simulate RAG pipeline
        return MockAnswer(
            text=f"This is the answer to: {request.question}",
            sources=["chunk-001", "chunk-002", "chunk-003"][:request.rerank_top_n],
            retrieval_k=request.rerank_top_n,
            embed_ms=150,
            search_ms=200,
            llm_ms=1200,
        )

# Create mock use case instance
mock_use_case = MockAskUseCase()

print("✅ Mock use case created!")

## 2. Ask Question Mutation

### 2.1 Implement ask_question mutation

Now let's implement the ask_question mutation with proper validation.

In [ ]:
@strawberry.type
class Mutation:
    @strawberry.mutation
    def ask_question(
        self,
        info,
        question: str,
        k: int = 5,
        document_id: Optional[strawberry.ID] = None,
    ) -> AnswerType:
        """
        Ask a question using GraphQL mutation.

        Args:
            info: GraphQL execution context
            question: Question to ask (required)
            k: Number of chunks to retrieve (default: 5)
            document_id: Optional document ID for chat mode

        Returns:
            Answer with text and sources
        """
        # 1. Validate inputs
        if not question or not question.strip():
            raise ValueError("question is required and cannot be empty")

        if k < 1 or k > 100:
            raise ValueError("k must be between 1 and 100")

        question = question.strip()

        if len(question) > 2000:
            raise ValueError("question too long (max 2000 characters)")

        # 2. Get tenant ID from mock context
        tenant_id = info.context.get("tenant_id", "demo-tenant-001")

        # 3. Get use case from context
        ask_use_case = info.context.get("ask_use_case", mock_use_case)
        
        if not ask_use_case:
            raise RuntimeError("Ask use case not available")

        # 4. Execute ask question use case
        from dataclasses import dataclass as dc
        
        @dc
        class MockRequest:
            tenant_id: str
            question: str
            document_id: Optional[str]
            rerank_top_n: int
        
        request_data = MockRequest(
            tenant_id=tenant_id,
            question=question,
            document_id=str(document_id) if document_id else None,
            rerank_top_n=k,
        )

        result = ask_use_case.execute(request_data)

        # 5. Convert to GraphQL type
        return AnswerType(
            text=result.text,
            sources=result.sources,
            retrieval_k=result.retrieval_k,
            embed_ms=result.embed_ms,
            search_ms=result.search_ms,
            llm_ms=result.llm_ms,
        )

print("✅ ask_question mutation defined successfully!")

### 2.2 Test the ask_question mutation

Let's test our mutation with a GraphQL query.

In [ ]:
# Create schema
schema = strawberry.Schema(query=Query, mutation=Mutation)

# Test mutation: ask a question
query = '''
mutation AskQuestion($question: String!, $k: Int, $documentId: ID) {
  askQuestion(question: $question, k: $k, documentId: $documentId) {
    text
    sources
    retrievalK
    embedMs
    searchMs
    llmMs
  }
}
'''

# Create mock context
mock_context = {
    "tenant_id": "demo-tenant-001",
    "ask_use_case": mock_use_case,
    "request": Mock(),
}

# Execute mutation
result = schema.execute_sync(
    query,
    variable_values={
        "question": "What is RAG?",
        "k": 5,
        "documentId": None,
    },
    context_value=mock_context,
)

# Display results
print("📄 GraphQL Mutation Results:")
if result.errors:
    print("Errors:")
    for error in result.errors:
        print(f"  - {error}")
else:
    print(result.data)

## 3. Error Handling

### 3.1 Test input validation

Let's test error handling with invalid inputs.

In [ ]:
# Test 1: Empty question (should fail)
print("Test 1: Empty question")
result = schema.execute_sync(
    query,
    variable_values={
        "question": "",
        "k": 5,
    },
    context_value=mock_context,
)
print(f"Errors: {result.errors}")

# Test 2: Invalid k value (should fail)
print("\nTest 2: Invalid k value")
result = schema.execute_sync(
    query,
    variable_values={
        "question": "What is RAG?",
        "k": 150,  # Invalid (max 100)
    },
    context_value=mock_context,
)
print(f"Errors: {result.errors}")

# Test 3: Too long question (should fail)
print("\nTest 3: Too long question")
result = schema.execute_sync(
    query,
    variable_values={
        "question": "x" * 3000,  # Too long (max 2000)
        "k": 5,
    },
    context_value=mock_context,
)
print(f"Errors: {result.errors}")

## 4. Performance Optimization

### 4.1 Caching Question Results

For repeated questions, we can cache the answers.

In [ ]:
from functools import lru_cache

# Cache decorator
@lru_cache(maxsize=10)
def _get_cached_answer(question: str, k: int) -> MockAnswer:
    """Cache answers for repeated questions."""
    return mock_use_case.execute(
        type('MockRequest', (), {
            'tenant_id': 'demo',
            'question': question,
            'document_id': None,
            'rerank_top_n': k,
        })
    )

# Test cache
question = "What is RAG?"
k = 5

# First call (cache miss)
result1 = _get_cached_answer(question, k)
print(f"First call (cache miss): {result1.text}")

# Second call (cache hit)
result2 = _get_cached_answer(question, k)
print(f"Second call (cache hit): {result2.text}")

# Check cache info
print(f"\nCache info: {_get_cached_answer.cache_info()}")

### 4.2 Async Mutation Example

For non-blocking operations, use async mutations.

In [ ]:
@strawberry.type
class AsyncMutation:
    @strawberry.mutation
    async def ask_question_async(
        self,
        info,
        question: str,
        k: int = 5,
    ) -> AnswerType:
        """
        Async mutation for non-blocking execution.
        """
        # Simulate async operation
        await asyncio.sleep(0.1)

        # Execute
        result = mock_use_case.execute(
            type('MockRequest', (), {
                'tenant_id': 'demo',
                'question': question,
                'document_id': None,
                'rerank_top_n': k,
            })
        )

        return AnswerType(
            text=result.text,
            sources=result.sources,
            retrieval_k=result.retrieval_k,
            embed_ms=result.embed_ms,
            search_ms=result.search_ms,
            llm_ms=result.llm_ms,
        )

print("✅ ask_question_async mutation defined!")

## 5. Practice Exercise

### Task: Implement document upload mutation

Create a mutation that:
1. Takes file content and filename as input
2. Validates file size (max 10MB)
3. Validates file type (pdf, docx, txt)
4. Returns created DocumentType
5. Includes error handling for invalid files

In [ ]:
# YOUR CODE HERE: Implement document upload mutation

@strawberry.type
class Mutation:
    # ... existing mutations ...

    @strawberry.mutation
    def upload_document(
        self,
        info,
        filename: str,
        content_type: str,
        size_bytes: int,
    ) -> DocumentType:
        """
        Upload a document.

        Args:
            filename: Document filename
            content_type: MIME type
            size_bytes: File size in bytes

        Returns:
            Created document
        """
        # TODO: Implement validation
        # - Validate file size (max 10MB)
        # - Validate content type

        # TODO: Implement upload logic
        # - Call document upload use case

        return DocumentType(
            id="doc-new",
            filename=filename,
            content_type=content_type,
            size_bytes=size_bytes,
            status=DocumentStatus.CREATED,
            created_at=datetime.utcnow(),
            updated_at=None,
        )

print("✅ upload_document mutation defined (TODO: implement validation)")

## 6. Quiz

### Question 1
What is the primary difference between GraphQL queries and mutations?

A) Queries execute in parallel, mutations execute sequentially
B) Mutations are for reading, queries are for writing
C) Queries are optional, mutations are required
D) Mutations have side effects, queries don't

**Answer:** D - Mutations have side effects.

---

### Question 2
How do you handle validation errors in GraphQL mutations?

A) Return error field in response
B) Raise ValueError with error message
C) Return null result
D) Log error and return success

**Answer:** B - Raise ValueError with error message.

---

### Question 3
What should ask_question mutation do besides generating an answer?

A) Only generate answer
B) Generate answer and save to query history
C) Generate answer, save history, and collect metrics
D) Only return sources

**Answer:** C - Generate answer, save history, collect metrics.

---

### Question 4
How do you optimize performance for repeated questions?

A) Always generate new answer
B) Cache answers by question and k
C) Use smaller k value
D) Stream responses

**Answer:** B - Cache answers for repeated questions.

---

### Question 5
Why is ask_question a mutation instead of a query?

A) It requires authentication
B) It has side effects (creates history, calls LLM)
C) It returns complex data
D) It uses external services

**Answer:** B - It has side effects (creates history, calls LLM).

## 7. Summary

In this notebook, you learned:

1. **GraphQL Mutations** - Operations that modify data
2. **Ask Question Mutation** - Complete implementation
3. **Error Handling** - Validation and error responses
4. **Performance Optimization** - Caching, async mutations
5. **Best Practices** - Use case integration, context management

### 🎯 Key Takeaways

- Mutations have side effects and execute sequentially
- Always validate inputs before processing
- Delegate business logic to use cases
- Use caching for repeated questions
- Handle errors gracefully with appropriate messages
- Extract tenant_id from request context for isolation

### 🚀 Next Steps

1. Implement the mutation code in `src/api/v1/graphql.py`
2. Test mutations using GraphQL Playground
3. Proceed to Phase 1.3: GraphQL Chat Session Resolvers

### 📚 Further Reading

- [Strawberry Mutations](https://strawberry.rocks/docs/mutations)
- [GraphQL Spec (Mutations)](https://spec.graphql.org/draft/#sec-Mutation)
- [GraphQL Best Practices](https://graphql.best practices/)

In [ ]:
# Print completion message
print("\n🎉 Congratulations!")
print("You've completed the GraphQL Ask Question Resolver tutorial!")
print("\n📝 Next: Implement this mutation in the actual codebase.")